In [6]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge, Dense,Concatenate,merge, multiply
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
import pickle

In [15]:
class ENTAILMENT():

    def __init__(self) : 
        S = Sentence_ent()
        S.load_dep()
        self.sentence = S

    def load_model_malstm(self):
        self.malstm_model = keras.models.load_model('MALSTM.h5')

    def get_parameters(self,sentence_A_processed,sentence_B_processed):
        features_function = K.function([left_input,right_input], [left_output,right_output])
        features = features_function([sentence_A_processed,sentence_B_processed])
        np_feature_difference = np.array(features)
        left_matrix = np_feature_difference[0]
        right_matrix = np_feature_difference[1]
        diff_features= np.abs(np.subtract(left_matrix, right_matrix))
        prod = np.multiply(left_matrix,right_matrix)
        self.extracted_features = np.concatenate([diff_features,prod], axis = 1)

    def load_model_entailment(self):
        self.dense_model = keras.models.load_model('CONTRADICTION_PREDICTION.h5')

    def entailment_prediction(self):
        pred_val =np.argmax(self.dense_model.predict(self.extracted_features))
        return pred_val

    def predict_value(self, A, B):
#         "CONTRADICTION" : 0 , "ENTAILMENT" : 1 ,"NEUTRAL" : 2
        S = self.sentence

        S.sentence = A
        A = S.preprocess()
        S.sentence = B
        B = S.preprocess()

        sentence_A_processed = A
        sentence_B_processed = B 
        self.load_model_malstm()
        self.get_parameters(sentence_A_processed,sentence_B_processed)
        self.load_model_entailment()
        entailment = self.entailment_prediction()
        return entailment

class Sentence_ent():
    
    def __init__(self):
        self.vocab_location = 'sick_vocab.pickle'
        self.embeddings_location = 'sick_embedding.pickle'
        # self.word2vec_location = '/Users/ebby/Documents/Fake news/MALSTM/GoogleNews-vectors-negative300.bin'
        
    def preprocess(self):
        text = self.sentence
        text = str(text)
        text = text.lower()
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)
        text = text.split()
        self.word_list = text
        self.to_index_and_pad()
        
        return self.index_rep
        
    def load_dep(self) : 
        file = open(self.vocab_location,"rb")
        self.vocabulary = pickle.load(file)
        file.close()
        file = open(self.embeddings_location, "rb")
        self.embeddings = pickle.load(file)
        file.close()
        print("Dependencies loaded!")
        
    def to_index_and_pad(self) : 
        
        self.index_rep = []
        for word in self.word_list : 
            if word in self.vocabulary : 
                self.index_rep.append(self.vocabulary[word])
        self.index_rep = pad_sequences([self.index_rep], maxlen=26)


In [16]:
A = "the dog was barking"
B = "the dog was running in the park"

In [17]:
e = ENTAILMENT()

Dependencies loaded!


In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge, Dense,Concatenate,merge, multiply
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras


Using TensorFlow backend.


In [2]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [5]:
print("Contradiction")
data = pd.read_csv('../main/SICK.txt', sep="\t", header=None)
data.columns = data.iloc[0]
data = data.reindex(data.index.drop(0))

EMBEDDING_FILE = '../main/GoogleNews-vectors-negative300.bin'


Contradiction


In [11]:
stops = set(stopwords.words('english'))
#test_df = #pd.read_csv('test file location', sep="\t", header=None)
vocabulary = dict()
inverse_vocabulary = ['<unk>']  
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
questions_cols = ['sentence_A' , 'sentence_B']

print("word2vec loaded!")

for dataset in [data]:
    for index, row in dataset.iterrows():
        for question in questions_cols:

            q2n = []  
            for word in text_to_word_list(row[question]):

                 # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                    #print(word)
                else:
                    q2n.append(vocabulary[word])
            dataset.set_value(index, question, q2n)

embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0 
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

word2vec loaded!


In [14]:
questions_cols = ['sentence_A' , 'sentence_B']
X_train = data[questions_cols]
Y_train = data['relatedness_score']
X_train = {'left': X_train.sentence_A, 'right': X_train.sentence_B}
Y_train = Y_train.values
max_seq_length = 26

In [15]:
for dataset, side in itertools.product([X_train], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

In [16]:
n_hidden = 50
gradient_clipping_norm = 1
batch_size = 64
n_epoch = 25
print("model 1")
def exponent_neg_manhattan_distance(left, right):
    return (K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True)))*4.0 +1

left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)
shared_lstm = LSTM(n_hidden)
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)
malstm_distance = Merge(mode=lambda x: ((K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)))*4.0 +1), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
malstm = Model([left_input, right_input], [malstm_distance])
optimizer = Adadelta(clipnorm=gradient_clipping_norm)
malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])


model 1


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [17]:
training_start_time = time()
malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch)
print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/25
9840/9840 [==============================] - 22s 2ms/step - loss: 1.6069 - mean_squared_error: 1.6069
Epoch 2/25
9840/9840 [==============================] - 21s 2ms/step - loss: 0.6733 - mean_squared_error: 0.6733
Epoch 3/25
9840/9840 [==============================] - 20s 2ms/step - loss: 0.5094 - mean_squared_error: 0.5094
Epoch 4/25
9840/9840 [==============================] - 20s 2ms/step - loss: 0.4355 - mean_squared_error: 0.4355
Epoch 5/25
9840/9840 [==============================] - 19s 2ms/step - loss: 0.3889 - mean_squared_error: 0.3889
Epoch 6/25
9840/9840 [==============================] - 19s 2ms/step - loss: 0.3564 - mean_squared_error: 0.3564
Epoch 7/25
9840/9840 [==============================] - 19s 2ms/step - loss: 0.3327 - mean_squared_error: 0.3327
Epoch 8/25
9840/9840 [==============================] - 19s 2ms/step - loss: 0.3144 - mean_squared_error: 0.3144
Epoch 9/25
9840/9840 [==============================] - 18s 2ms/step - loss: 0.2983 - mean_squar

In [19]:
malstm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 300)      691200      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 50)           70200       embedding_1[0][0]                
          

In [40]:
malstm.input

[<tf.Tensor 'input_1:0' shape=(?, 26) dtype=int32>,
 <tf.Tensor 'input_2:0' shape=(?, 26) dtype=int32>]

In [38]:
l3 = K.function([malstm.layers[0].input, K.learning_phase()], [malstm.layers[3].output])

AttributeError: Layer lstm_1 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [139]:
intermediate_layer_model = Model(inputs=[malstm.input[0]], outputs=[malstm.layers[4].get_input_at(0)[0]])
# intermediate_output = intermediate_layer_model.predict(data)

In [52]:
malstm.get_layer('lstm_1').get_input_at(0).output

AttributeError: 'Tensor' object has no attribute 'output'

In [57]:
malstm.input[1]

<tf.Tensor 'input_2:0' shape=(?, 26) dtype=int32>

In [74]:
dataset["right"][10]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 27, 28,  3,
       35, 30, 31,  5, 32, 11,  9,  5, 34], dtype=int32)

In [134]:
left = dataset["left"][10:15]
right = dataset["right"][10:15]

In [141]:
intermediate_output = intermediate_layer_model.predict([left])

In [133]:
malstm.layers[4].get_input_at(0)[2]

IndexError: list index out of range

In [91]:
x.shape

(5, 26)

In [88]:
np.array(list(x))

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 27, 28,  3,
       35, 30, 31,  5, 32, 11,  9,  5, 34], dtype=int32)

In [144]:
intermediate_output

array([[ -2.77754222e-03,   2.14841729e-03,   1.91244471e-05,
         -1.94815442e-01,  -2.77200460e-01,   3.04105133e-01,
          2.64453322e-01,   5.65008521e-02,   1.28314563e-03,
         -1.74586892e-01,   1.92945302e-01,   5.40861845e-01,
         -8.18348853e-05,   5.82041964e-02,  -4.72569019e-02,
          2.68744737e-01,  -5.25615104e-02,  -7.28903860e-02,
          3.28205675e-01,   2.23158300e-01,  -3.81865129e-02,
         -4.95558558e-03,  -9.65355357e-05,  -6.66834693e-03,
          1.67123362e-01,  -4.07805890e-01,   3.63875516e-02,
          7.88677542e-04,   1.32974043e-01,   4.33675665e-03,
          1.30774617e-01,  -1.38347282e-03,   4.17650212e-03,
         -3.93529087e-01,   2.44918928e-01,  -1.27814621e-01,
         -1.87294915e-01,  -1.85248524e-01,  -4.95179780e-02,
          2.55396720e-02,   3.44460859e-04,  -0.00000000e+00,
         -1.87056642e-02,   2.82574862e-01,  -4.14009318e-02,
          5.48415817e-03,   6.80437486e-04,   4.57161099e-01,
        

In [261]:
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 26, 300)           691200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
Total params: 761,400
Trainable params: 70,200
Non-trainable params: 691,200
_________________________________________________________________


In [116]:
malstm.layers[1]

In [146]:

# serialize model to JSON
model_json = intermediate_layer_model.to_json()
with open("malstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
intermediate_layer_model.save_weights("malstm_weights.h5")



In [157]:
from keras.models import model_from_json
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge,merge,Dense,Dropout
from keras.optimizers import Adadelta,Adam


In [154]:
# load json and create model
json_file = open('malstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("malstm_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [150]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 26, 300)           691200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
Total params: 761,400
Trainable params: 70,200
Non-trainable params: 691,200
_________________________________________________________________


In [266]:
# loaded_model.predict([left])
left_matrix = intermediate_layer_model.predict([dataset['left']])
right_matrix = intermediate_layer_model.predict(dataset['right'])
diff_features = np.abs(np.subtract(left_matrix, right_matrix))
prod = np.multiply(left_matrix,right_matrix)
extracted_features = np.concatenate([diff_features,prod], axis = 1)

In [272]:
extracted_features.shape

(9840, 100)

In [278]:
model2 = Sequential()
model2.add(Dense(200, input_shape=(100,),kernel_initializer='he_normal', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(100, kernel_initializer='he_normal', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(50, kernel_initializer='he_normal', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(3, activation='softmax'))
model2.compile(Adam(lr = 0.001),'categorical_crossentropy', metrics=['accuracy'])
# model.fit(extracted_features, y_TRAIN_cat.values,batch_size=16,epochs=10,verbose=1)
# print("Entailmet model finished training")

In [160]:
l = intermediate_layer_model.predict(dataset["left"])

In [161]:
r = intermediate_layer_model.predict(dataset["right"])

In [242]:
i2  = [n(l)

9840

In [241]:
len(list(l[0]) + list(l[0]))

100

In [243]:
for i in range(len(l)) : 
    i2 += [list(l[i]) + list(r[i])]

In [244]:
len(i2[0])

100

In [210]:
Y_train1 = data['entailment_label']

In [211]:
y2 = pd.get_dummies(Y_train1)

In [280]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 200)               20200     
_________________________________________________________________
dropout_76 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_103 (Dense)            (None, 100)               20100     
_________________________________________________________________
dropout_77 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 50)                5050      
_________________________________________________________________
dropout_78 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_105 (Dense)            (None, 3)                 153       
Total para

In [247]:
# list(y2.values[0])
i3 = np.array(i2)

In [248]:
i3.shape

(9840, 100)

In [281]:
model2.fit(extracted_features, y2.values,batch_size=16,epochs=10,verbose=1)

Epoch 1/10
9840/9840 [==============================] - 5s 495us/step - loss: 0.5724 - acc: 0.7679
Epoch 2/10
9840/9840 [==============================] - 3s 352us/step - loss: 0.4081 - acc: 0.8486
Epoch 3/10
9840/9840 [==============================] - 3s 351us/step - loss: 0.3937 - acc: 0.8568
Epoch 4/10
9840/9840 [==============================] - 4s 358us/step - loss: 0.3875 - acc: 0.8588
Epoch 5/10
9840/9840 [==============================] - 3s 348us/step - loss: 0.3866 - acc: 0.8589
Epoch 6/10
9840/9840 [==============================] - 3s 354us/step - loss: 0.3822 - acc: 0.8612
Epoch 7/10
9840/9840 [==============================] - 3s 350us/step - loss: 0.3808 - acc: 0.8610
Epoch 8/10
9840/9840 [==============================] - 3s 348us/step - loss: 0.3773 - acc: 0.8620
Epoch 9/10
9840/9840 [==============================] - 3s 355us/step - loss: 0.3720 - acc: 0.8620
Epoch 10/10
9840/9840 [==============================] - 3s 353us/step - loss: 0.3721 - acc: 0.8642


In [263]:
##HAVE TO TEST THE MODEL
from sklearn.model_selection import cross_val_score
from sklearn import metrics 
from sklearn.model_selection import StratifiedKFold


# scores = cross_val_score(stance, input_np, y_np, cv=5, scoring="accuracy")
seed = 1
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)

cvscores = []
for train, test in kfold.split(i3, Y_train1):
  # create model
    model2 = Sequential()
    model2.add(Dense(200, input_shape=(100,),kernel_initializer='he_normal', activation='relu'))
    model2.add(Dropout(0.2))
    model2.add(Dense(100, kernel_initializer='he_normal', activation='relu'))
    model2.add(Dropout(0.2))
    model2.add(Dense(50, kernel_initializer='he_normal', activation='relu'))
    model2.add(Dropout(0.2))
    model2.add(Dense(3, activation='softmax'))
    model2.compile(Adam(lr = 0.001),'categorical_crossentropy', metrics=['accuracy'])
    # Fit the model
#     stance.fit(input_np[train], y_cat[train], epochs=epochs, verbose=1, batch_size=batch_size)
    model2.fit(i3[train], y2.values[train],batch_size=16,epochs=90,verbose=1)
    # evaluate the model
    scores = model2.evaluate(i3[test], y2.values[test], verbose=0)
    print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/90
7379/7379 [==============================] - 4s 505us/step - loss: 0.9647 - acc: 0.5684
Epoch 2/90
7379/7379 [==============================] - 3s 342us/step - loss: 0.9424 - acc: 0.5689
Epoch 3/90
7379/7379 [==============================] - 3s 351us/step - loss: 0.9147 - acc: 0.5728
Epoch 4/90
7379/7379 [==============================] - 3s 339us/step - loss: 0.8911 - acc: 0.5776
Epoch 5/90
7379/7379 [==============================] - 3s 344us/step - loss: 0.8867 - acc: 0.5774
Epoch 6/90
7379/7379 [==============================] - 3s 351us/step - loss: 0.8726 - acc: 0.5810
Epoch 7/90
7379/7379 [==============================] - 3s 356us/step - loss: 0.8693 - acc: 0.5772
Epoch 8/90
7379/7379 [==============================] - 3s 342us/step - loss: 0.8590 - acc: 0.5840
Epoch 9/90
7379/7379 [==============================] - 3s 343us/step - loss: 0.8383 - acc: 0.5877
Epoch 10/90
7379/7379 [==============================] - 3s 345us/step - loss: 0.8159 - acc: 0.5934
Epoch 11/

7379/7379 [==============================] - 3s 348us/step - loss: 0.5506 - acc: 0.7577
Epoch 84/90
7379/7379 [==============================] - 3s 346us/step - loss: 0.5351 - acc: 0.7696
Epoch 85/90
7379/7379 [==============================] - 3s 343us/step - loss: 0.5466 - acc: 0.7585
Epoch 86/90
7379/7379 [==============================] - 3s 351us/step - loss: 0.5319 - acc: 0.7688
Epoch 87/90
7379/7379 [==============================] - 3s 345us/step - loss: 0.5308 - acc: 0.7672
Epoch 88/90
7379/7379 [==============================] - 3s 351us/step - loss: 0.5395 - acc: 0.7706
Epoch 89/90
7379/7379 [==============================] - 3s 348us/step - loss: 0.5367 - acc: 0.7668
Epoch 90/90
7379/7379 [==============================] - 3s 358us/step - loss: 0.5295 - acc: 0.7704
acc: 77.85%
Epoch 1/90
7380/7380 [==============================] - 4s 521us/step - loss: 0.9682 - acc: 0.5667
Epoch 2/90
7380/7380 [==============================] - 3s 355us/step - loss: 0.9444 - acc: 0.5687
Ep

7380/7380 [==============================] - 3s 342us/step - loss: 0.5677 - acc: 0.7485
Epoch 76/90
7380/7380 [==============================] - 3s 344us/step - loss: 0.5553 - acc: 0.7550
Epoch 77/90
7380/7380 [==============================] - 3s 352us/step - loss: 0.5425 - acc: 0.7686
Epoch 78/90
7380/7380 [==============================] - 3s 351us/step - loss: 0.5577 - acc: 0.7497
Epoch 79/90
7380/7380 [==============================] - 3s 362us/step - loss: 0.5555 - acc: 0.7595
Epoch 80/90
7380/7380 [==============================] - 3s 340us/step - loss: 0.5548 - acc: 0.7599
Epoch 81/90
7380/7380 [==============================] - 2s 336us/step - loss: 0.5464 - acc: 0.7637
Epoch 82/90
7380/7380 [==============================] - 3s 346us/step - loss: 0.5505 - acc: 0.7570
Epoch 83/90
7380/7380 [==============================] - 3s 341us/step - loss: 0.5548 - acc: 0.7583
Epoch 84/90
7380/7380 [==============================] - 3s 343us/step - loss: 0.5428 - acc: 0.7615
Epoch 85/90


7380/7380 [==============================] - 3s 346us/step - loss: 0.5622 - acc: 0.7501
Epoch 68/90
7380/7380 [==============================] - 3s 349us/step - loss: 0.5549 - acc: 0.7549
Epoch 69/90
7380/7380 [==============================] - 3s 350us/step - loss: 0.5633 - acc: 0.7554
Epoch 70/90
7380/7380 [==============================] - 3s 349us/step - loss: 0.5690 - acc: 0.7473
Epoch 71/90
7380/7380 [==============================] - 3s 352us/step - loss: 0.5519 - acc: 0.7592
Epoch 72/90
7380/7380 [==============================] - 3s 348us/step - loss: 0.5527 - acc: 0.7564
Epoch 73/90
7380/7380 [==============================] - 3s 345us/step - loss: 0.5456 - acc: 0.7623
Epoch 74/90
7380/7380 [==============================] - 3s 350us/step - loss: 0.5510 - acc: 0.7607
Epoch 75/90
7380/7380 [==============================] - 3s 349us/step - loss: 0.5458 - acc: 0.7576
Epoch 76/90
7380/7380 [==============================] - 3s 349us/step - loss: 0.5503 - acc: 0.7538
Epoch 77/90


In [334]:
# serialize model to JSON
model2_json = model2.to_json()
with open("contra.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model2.save_weights("contra_weights.h5")

In [335]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 200)               20200     
_________________________________________________________________
dropout_76 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_103 (Dense)            (None, 100)               20100     
_________________________________________________________________
dropout_77 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 50)                5050      
_________________________________________________________________
dropout_78 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_105 (Dense)            (None, 3)                 153       
Total para

In [286]:
a = np.array(a)

In [290]:
a = a.map({"hello": 1, "world": 2})

AttributeError: 'list' object has no attribute 'map'

In [311]:
a = open('./sick_vocab.pickle', "rb")

In [312]:
import pickle

In [313]:
vocab = pickle.load(a)

In [332]:
"over" in vocab

True

In [307]:
def create_dataframe(list_heading,list_crawled):
    ls = []
    c =[]
    news_dataset =pd.DataFrame(columns=["sentence_A","sentence_B"])
    for heading in range(0,len(list_heading)):
        #print(heading,list_heading[heading])
        crawled_data = pd.DataFrame()
        for crawl in list_crawled[heading]:
            ls.append(list_heading[heading])
            c.append(crawl)
    news_dataset["sentence_A"]=ls
    news_dataset["sentence_B"]=c
    return news_dataset

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    return text

# Prepare embedding
def preprocess(text):
    word_list = text_to_word_list(text)
    ind = []
    for word in word_list : 
        if(word in vocab) : 
            ind.append(vocab[word])
    return ind 

In [319]:
v = "Two children are lying in the snow and are making snow angels"

In [320]:
q = text_to_word_list(v)

In [321]:
ind = []
for word in q : 
    if(word in vocab) : 
        ind.append(vocab[word])
    

In [333]:
pad_sequences([ind], maxlen=26)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 27, 14,
        19, 85,  5, 11, 84, 19, 83, 84, 82]], dtype=int32)

In [327]:
stops = set(stopwords.words('english'))

In [331]:
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
fro scik